In [ ]:
# pip install https://github.com/yorak/VeRyPy/zipball/master

from verypy.cvrp_io import read_TSPLIB_CVRP
from verypy.classic_heuristics.parallel_savings import parallel_savings_init
from verypy.util import sol2routes, sol2edgeset

In [ ]:
import os 
import sys
import random
import argparse
from datetime import datetime
import warnings
warnings.simplefilter('ignore')


import gym
from gym import spaces

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import pickle

In [ ]:
train_data_path = "./data/train/"+ f"train_graphs_10000x10.pickle"
test_data_path = "./data/test/"+ f"test_graphs_dynamic_10000x10.pickle"

#with open(train_data_path, 'rb') as train_handle:
    #load_train_graphs = pickle.load(train_handle)
    
with open(test_data_path, 'rb') as test_handle:
    load_test_graphs = pickle.load(test_handle)  

In [ ]:
# A peak into generated instances

#print(f"# of Training graphs: {len(load_train_graphs)}")
print(f"# of Test graphs: {len(load_test_graphs)}")

n = 10
#print("\n", load_train_graphs[n]["node_features"])
print("\n", load_test_graphs[n]["node_features"])

In [ ]:
def sol2length(solution, distance_matrix):
    tour_len = 0
    sol_edges = [(solution[i], solution[i+1]) for i in range(len(solution) - 1)]
    for edge in sol_edges:
        tour_len += distance_matrix[edge[0], edge[1]]
    return float(tour_len)


def solve_graph(graph_data):
    #Derive graph data in required format
    distances = graph_data["edge_features"]
    demands = graph_data["node_features"][:, 2]
    solution = parallel_savings_init(D=distances, d=demands, C=1)
    cost = sol2length(solution, distances)
    return solution, cost

In [ ]:
test = load_test_graphs[10]
solution, cost = solve_graph(test)

for route_idx, route in enumerate(sol2routes(solution)):
    print("Route #%d : %s"%(route_idx+1, route))

In [ ]:
print(solution)
print(cost)

In [ ]:
# Evaluating on entire dataset and storing results in a df

graph_solutions = []

for idx, graph in enumerate(load_test_graphs):
    if idx%500 == 0:
        print(f"solved {idx} graphs")
    solution, cost = solve_graph(graph)
    graph_solution = {"idx": idx, "solution": solution, "cost": cost}
    graph_solutions.append(graph_solution)
    

eval_df_cols = graph_solution.keys()
eval_df = pd.DataFrame(graph_solutions, columns = eval_df_cols)

In [ ]:
eval_df.head()

In [ ]:
res_path = "./trained_models/results/"
eval_df.to_csv(res_path + f"eval_df_cls_d10", index=False)